In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import csv
import numpy as np
import pandas as pd
import os

In [2]:
filename = 'websites/alexa_websites.csv' #file path that will contain the csv file

In [3]:
def getWebsites(element): #returns True if element is a listed websites
    return re.search('<a href="/siteinfo/', str(element))

In [4]:
def getCategories(element): #returns True if element is a subcategory
    return re.search('<a href="/topsites/category/Top/', str(element))

In [8]:
#getCategoryPath recursively goes through subcategories and runs getWebsitesList only for the deepest subcategories

def getCategoryPath(current_path): 
    print(current_path)
    
    try: #trying to access current url
        htmltext = urllib.request.urlopen("https://www.alexa.com/topsites/category/Top" + current_path).read()
    except: #catches any error in opening current url
        print("Error loading page")
        return
        
    soup = BeautifulSoup(htmltext)
    
    if soup.find(string=re.compile("No sites for this category.")) != None: #do nothing if the smallest sub-category contains no websites
        return
    
    texts = soup.findAll(href=True)

    categories_html = filter(getCategories, texts) #contains all categories with some syntax noise
    categories = []

    for text in categories_html: #adds cleaned category string to the categories array
        temp = str(text.encode("utf-8"))
        if current_path == "":
            categories.append(re.search('Top/(.+?)"', temp).group(1))
        else:
            categories.append(re.search(current_path + '/(.+?)"', temp).group(1))
                              
    if current_path != "" and soup.find(string=re.compile("Sub-Categories \(")) == None: #get the list of top websites of the deepest sub-category
        getWebsitesList(current_path)
                              
    for category in categories: #recurse until it reaches the deepest sub-category
        if category == 'Adult':
            continue
        getCategoryPath(current_path + "/" + category)
    

In [9]:
#getWebsitesList inserts top 50 websites of the category_path into csv file

def getWebsitesList(category_path):
    try:
        htmltext = urllib.request.urlopen("https://www.alexa.com/topsites/category/Top" + category_path).read()
    except:
        print("Error loading page")
        return

    soup = BeautifulSoup(htmltext)
    texts = soup.findAll(href=True)

    websites = filter(getWebsites, texts)
    result = []

    for text in websites:
        temp = str(text.encode("utf-8"))
        result.append([re.search('siteinfo/(.+?)"', temp).group(1), category_path])
    result_np = np.asarray(result)

    with open(filename, 'a') as f:
        if not os.path.exists(filename):
            pd.DataFrame(result_np).to_csv(f, header=True)
        else:
            pd.DataFrame(result_np).to_csv(f, header=False)

In [ ]:
if os.path.exists(filename):
    os.remove(filename)
getCategoryPath("")


/Arts
/Arts/Animation
/Arts/Animation/Anime
/Arts/Animation/Anime/Characters
/Arts/Animation/Anime/Clubs_and_Organizations
/Arts/Animation/Anime/Collectibles
/Arts/Animation/Anime/Collectibles/Cels
/Arts/Animation/Anime/Collectibles/Models_and_Figures
/Arts/Animation/Anime/Collectibles/Models_and_Figures/Action_Figures
/Arts/Animation/Anime/Collectibles/Models_and_Figures/Action_Figures/Gundam
/Arts/Animation/Anime/Collectibles/Models_and_Figures/Action_Figures/Zoids
/Arts/Animation/Anime/Collectibles/Models_and_Figures/Models
/Arts/Animation/Anime/Collectibles/Models_and_Figures/Models/Gundam
/Arts/Animation/Anime/Collectibles/Shitajiki
/Arts/Animation/Anime/Creators
/Arts/Animation/Anime/Creators/Anno,_Hideaki
/Arts/Animation/Anime/Creators/Ikuhara,_Kunihiko
/Arts/Animation/Anime/Creators/Miyazaki,_Hayao
/Arts/Animation/Anime/Creators/Studios
/Arts/Animation/Anime/Creators/Studios/Studio_Ghibli
/Arts/Animation/Anime/Creators/Studios/Studio_Ghibli/Titles
/Arts/Animation/Anime/Distrib